In [16]:
# Instalações necessárias das apis do google para rodar este projeto

%pip -q install google-genai
!pip install -q google-adk

In [17]:
# Configurar a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [18]:
# Configurar o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [47]:
# Imports necessários
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [53]:
# 🧪 Carboninho - Assistente Virtual de Química
print("🧬 Oi! Sou o Carboninho, seu assistente virtual de laboratório químico! 👨‍🔬🔬")
print("\n")

# --- Funções auxiliares ---
from IPython.display import Markdown
import textwrap

def to_markdown(text):
    text = text.replace('•', '  *')
    text = text.replace("## ", "### ")
    text = text.replace("# ", "## ")
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def call_agent(agent, message_text):
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text:
                    final_response += part.text + "\n"
    return f"## 🧪 Resultado do Carboninho\n\n{final_response.strip()}"

# --- Definição dos agentes ---

def agente_busca_quimica(pergunta):
    agente = Agent(
        name="agente_busca_quimica",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um especialista em química. Use o google_search para buscar informações detalhadas e precisas sobre:
        '{pergunta}'. Foque em propriedades, reações, mecanismos, aplicações, etc.
        """,
        description="Agente que busca informações químicas no Google",
        tools=[google_search]
    )
    return call_agent(agente, f"🔍 Buscar informações sobre: {pergunta}")

def agente_explica_conceito(conceito):
    agente = Agent(
        name="agente_explica_conceito",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um professor de química. Explique o conceito '{conceito}' de forma clara, com exemplos do cotidiano se possível.
        """,
        description="Agente explicador de conceitos químicos",
        tools=[]
    )
    return call_agent(agente, f"📘 Explicar conceito: {conceito}")

def agente_planeja_experimento_virtual(objetivo, nivel):
    agente = Agent(
        name="agente_planeja_experimento_virtual",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um especialista em laboratório virtual. Ajude um estudante de nível '{nivel}' a planejar um experimento virtual
        com objetivo '{objetivo}'. Sugira simuladores, passo a passo e cuidados de segurança.
        """,
        description="Planejador de experimentos virtuais de química",
        tools=[google_search]
    )
    return call_agent(agente, f"🧪 Planejar experimento: objetivo={objetivo}, nível={nivel}")

def agente_calcula_quimica():
    return Agent(
        name="agente_calcula_quimica",
        model="gemini-2.0-flash",
        instruction="""
        Você resolve cálculos químicos: molaridade, pH, estequiometria, etc. Gere código Python comentado para resolver.
        Peça mais dados se necessário.
        """,
        description="Calculadora de química com código Python",
        tools=[]
    )

def agente_seguranca_laboratorio(pergunta, composto=None, procedimento=None):
    agente = Agent(
        name="agente_seguranca_laboratorio",
        model="gemini-2.0-flash",
        instruction=f"""
        Especialista em segurança de laboratório. Responda à pergunta '{pergunta}' com foco em segurança.
        Inclua informações sobre '{composto}' e '{procedimento}' se fornecidos.
        Liste EPIs, riscos e ações de emergência.
        """,
        description="Agente de segurança de laboratório químico",
        tools=[google_search]
    )
    entrada = f"🔐 {pergunta}"
    if composto:
        entrada += f"\n🧪 Composto: {composto}"
    if procedimento:
        entrada += f"\n🧫 Procedimento: {procedimento}"
    return call_agent(agente, entrada)

def agente_resumo_final(nome_usuario, interacoes):
    agente = Agent(
        name="agente_resumo_final",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um assistente educacional que vai gerar um resumo final da sessão para o usuário '{nome_usuario}'.
        Com base nas interações registradas abaixo, gere um texto que:
        - Recapitule os principais tópicos abordados;
        - Destaque aprendizados ou respostas recebidas;
        - Sugira próximos passos ou tópicos relacionados para estudo futuro.
        Utilize emojis educativos e estrutura de tópicos.
        """,
        description="Resumo final da sessão com dicas e revisão de conteúdo",
        tools=[]
    )
    entrada = f"📘 Interações da sessão:\n{interacoes}"
    return call_agent(agente, entrada)

def calcular(agente, pergunta):
    return call_agent(agente, f"🧮 Pergunta de cálculo: {pergunta}")

# --- Fluxo principal ---

def run_carboninho():
    nome = input("Informe seu nome: ").strip()
    interacoes_usuario = ""

    while True:
        print()  # linha em branco para espaçar
        nivel = input("📊 Nível de conhecimento em química (iniciante/médio/avançado) ou 'sair': ").lower().strip()
        if nivel == "sair":
            print(f"\n🔬 Encerramento: Foi ótimo conversar com você, {nome}! Até a próxima! 👋")
            break

        objetivo = input("\n🎯 Qual o seu objetivo principal hoje? ").strip()

        while True:
            pergunta = input("\n💬 Faça sua pergunta sobre química (ou digite 'sair' para encerrar): ").strip()
            if pergunta.lower() == "sair":
                print(f"\n📚 Resumo da sessão com {nome}:\n")
                resumo = agente_resumo_final(nome, interacoes_usuario)
                display(to_markdown(resumo))
                print(f"\n🔬 Foi ótimo conversar com você, {nome}! Até a próxima! 👋")
                return

            p = pergunta.lower()
            if any(x in p for x in ["calcular", "cálculo", "molaridade", "estequiometria", "ph"]):
                resposta = calcular(agente_calcula_quimica(), pergunta)
            elif any(x in p for x in ["segurança", "epi", "risco", "precaução", "emergência"]):
                resposta = agente_seguranca_laboratorio(pergunta)
            elif any(x in p for x in ["experimento", "laboratório virtual", "simulação", "planejar experimento"]):
                resposta = agente_planeja_experimento_virtual(objetivo, nivel)
            elif any(x in p for x in ["explicar", "conceito", "definição", "o que é", "como funciona"]):
                resposta = agente_explica_conceito(pergunta)
            else:
                resposta = agente_busca_quimica(pergunta)

            interacoes_usuario += f"\n🔎 Pergunta: {pergunta}\n📥 Resposta: {resposta}\n"
            display(to_markdown(resposta))

# --- Execução ---
if __name__ == "__main__":
    run_carboninho()

🧬 Oi! Sou o Carboninho, seu assistente virtual de laboratório químico! 👨‍🔬🔬


Informe seu nome: Kaique

📊 Nível de conhecimento em química (iniciante/médio/avançado) ou 'sair': iniciante

🎯 Qual o seu objetivo principal hoje? aprendizado técnico

💬 Faça sua pergunta sobre química (ou digite 'sair' para encerrar): como realizar uma titulação?


> #### 🧪 Resultado do Carboninho
> 
> Para fornecer informações detalhadas sobre como realizar uma titulação, podemos explorar os seguintes aspectos:
> 
>  *  **O que é uma titulação e seus princípios básicos?** Qual a reação química envolvida e como ela é utilizada para determinar a concentração de uma solução desconhecida?
>  *  **Quais são os tipos de titulação existentes?** Titulação ácido-base, redox, complexométrica, precipitação, etc. Quais as diferenças e aplicações de cada uma?
>  *  **Quais são os materiais e equipamentos necessários?** Bureta, erlenmeyer, pipeta, balão volumétrico, indicador, agitador magnético, etc. Como preparar e calibrar esses materiais?
>  *  **Como escolher o indicador correto?** Qual a faixa de pH em que o indicador muda de cor e como isso se relaciona com o ponto de equivalência da titulação?
>  *  **Como realizar o procedimento experimental passo a passo?** Preparação das soluções, montagem do sistema de titulação, adição do titulante, observação da mudança de cor do indicador, leitura do volume gasto, etc.
>  *  **Como calcular a concentração da solução desconhecida?** Utilização da estequiometria da reação e do volume gasto do titulante para determinar a concentração.
>  *  **Quais são os erros comuns em titulações e como evitá-los?** Erros de leitura, erros de calibração, erros na preparação das soluções, etc.
>  *  **Quais são as aplicações da titulação em diferentes áreas?** Química, biologia, medicina, alimentos, meio ambiente, etc.
> 
>  
> A titulação é uma técnica de laboratório quantitativa utilizada para determinar a concentração desconhecida de uma substância (analito) em uma solução. O procedimento envolve a adição controlada de um reagente de concentração conhecida (titulante) que reage estequiometricamente com o analito. A titulação é finalizada quando a reação entre o titulante e o analito é completa, o que é sinalizado por uma mudança observável, como a mudança de cor de um indicador.
> 
>  ##### Tipos de Titulação:
>  *  **Ácido-base (Neutralização):** Determina a concentração de ácidos ou bases usando a reação de neutralização. O ponto final é indicado por um indicador de pH. Exemplo: Determinação da concentração de ácido clorídrico com hidróxido de sódio.
>  *  **Redox (Oxidação-redução):** Envolve reações de transferência de elétrons entre o titulante e o analito. Exemplo: Determinação da concentração de vitamina C.
>  *  **Complexométrica:** Baseia-se na formação de complexos entre íons metálicos e um agente complexante, como o EDTA. Exemplo: Determinação da dureza da água medindo íons de cálcio e magnésio.
>  *  **Precipitação:** O titulante reage com o analito formando um precipitado insolúvel. Exemplo: Determinação de cloreto por titulação com nitrato de prata.
> 
>  ##### Materiais e Equipamentos Necessários:
>  *   **Bureta:** Tubo de vidro graduado com uma torneira para dispensar o titulante de forma controlada.
>  *   **Erlenmeyer:** Frasco para conter o analito e permitir a mistura durante a titulação.
>  *   **Pipeta:** Utilizada para medir volumes precisos do analito.
>  *   **Béquer:** Usado para preparar e transferir soluções.
>  *   **Suporte Universal e Garra:** Para fixar e manter a bureta na posição vertical.
>  *   **Agitador Magnético:** Garante a mistura homogênea da solução durante a titulação.
>  *   **Indicador:** Substância que muda de cor para indicar o ponto final da titulação.
>  *   **Soluções:**
>  *   **Titulante:** Solução de concentração conhecida.
>  *  **Analito (Titulado):** Solução de concentração desconhecida.
>  *   **Padrão Primário:** Substância de alta pureza utilizada para padronizar o titulante.
> 
>  ##### Procedimento Experimental:
>  1. **Preparação das soluções:** Preparar o titulante com concentração conhecida e dissolver o analito em um solvente adequado.
>  2. **Montagem:** Fixar a bureta no suporte universal e posicionar o erlenmeyer com o analito sob a bureta.
>  3. **Adição do indicador:** Adicionar algumas gotas do indicador apropriado ao analito no erlenmeyer.
>  4. **Titulação:** Abrir a torneira da bureta e adicionar o titulante lentamente ao analito, agitando continuamente o erlenmeyer.
>  5. **Ponto final:** Observar a mudança de cor do indicador e fechar a torneira quando a mudança de cor persistir, indicando o ponto final da titulação.
>  6. **Leitura:** Registrar o volume de titulante gasto na bureta.
>  7. **Repetição:** Repetir o processo para obter resultados consistentes e precisos.
> 
>  ##### Escolha do Indicador:
>  *   O indicador deve mudar de cor próximo ao ponto de equivalência da titulação.
>  *   A faixa de pH de transição do indicador deve corresponder à variação brusca de pH que ocorre na titulação.
>  *   Exemplos de indicadores comuns incluem fenolftaleína (incolor em meio ácido, rosa em meio básico) e alaranjado de metila (vermelho em meio ácido, amarelo em meio básico).
> 
>  ##### Cálculos na Titulação:
>  1. **Equacionar a reação:** Escrever a equação química balanceada da reação entre o titulante e o analito.
>  2. **Calcular os mols do titulante:** Utilizar a concentração e o volume do titulante para calcular o número de mols.
>  3. **Determinar os mols do analito:** Usar a estequiometria da reação para determinar o número de mols do analito.
>  4. **Calcular a concentração do analito:** Dividir o número de mols do analito pelo volume da solução do analito.
> 
>  ##### Erros Comuns e Como Evitá-los:
>  *  **Erro de paralaxe:** Ler o volume na bureta no nível correto dos olhos.
>  *   **Bolhas na bureta:** Eliminar bolhas de ar na bureta antes de iniciar a titulação.
>  *   **Indicador inadequado:** Escolher um indicador com faixa de viragem apropriada para a titulação.
>  *   **Não agitar:** Agitar continuamente a solução durante a titulação para garantir a homogeneidade.
>  *   **Calibração:** Verificar a calibração dos equipamentos volumétricos.
>  *  **Pureza dos reagentes:** Utilizar reagentes com pureza conhecida.
> 
>  ##### Aplicações da Titulação:
>  *   **Química:** Determinação da concentração de soluções ácidas e básicas.
>  *   **Alimentos:** Análise da acidez em alimentos e bebidas.
>  *   **Farmacêutica:** Quantificação de princípios ativos em medicamentos.
>  *   **Ambiental:** Monitoramento da qualidade da água, determinando a concentração de poluentes.
>  *   **Clínica:** Análise de componentes em amostras biológicas.
> 
>  ##### Vantagens da Titulação:
>  *   **Simplicidade:** Técnica fácil de executar e entender.
>  *   **Custo-benefício:** Requer equipamentos de laboratório básicos.
>  *   **Precisão:** Resultados precisos quando realizada corretamente.
>  *   **Versatilidade:** Aplicável em diversas áreas e tipos de amostras.


💬 Faça sua pergunta sobre química (ou digite 'sair' para encerrar): sair

📚 Resumo da sessão com Kaique:



> #### 🧪 Resultado do Carboninho
> 
> 👋 Olá, Kaique! Aqui está um resumo da nossa sessão sobre titulação:
> 
> **Tópicos Abordados:**
> *   🧪 O que é titulação e seus princípios básicos.
> *   ⚗️ Tipos de titulação (ácido-base, redox, complexométrica, precipitação).
> *   🔬 Materiais e equipamentos necessários (bureta, erlenmeyer, pipeta, etc.).
> *   🌈 Como escolher o indicador correto.
> *   📝 Procedimento experimental passo a passo.
> *   🧮 Cálculos para determinar a concentração da solução desconhecida.
> *   ⚠️ Erros comuns e como evitá-los.
> *    Aplicações da titulação em diversas áreas.
> 
> **Próximos Passos:**
> *   📚 Explore mais sobre os diferentes tipos de titulação e suas aplicações específicas.
> *   🔍 Aprofunde-se nos cálculos estequiométricos envolvidos na titulação.
> *   👩‍🔬 Considere realizar experimentos práticos de titulação para consolidar o aprendizado.


🔬 Foi ótimo conversar com você, Kaique! Até a próxima! 👋
